## Builder‘s Guide

### Layers and Modules

- Mulitple layers are combined into modules

In [8]:
import torch
from torch import nn
from torch.nn import functional as F

In [9]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand((2, 10))
net(X).shape

torch.Size([2, 10])

#### A Custom Module

In [10]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
net = MLP()
net(X).shape

torch.Size([2, 10])

#### The Sequential Module

- A method to append modules one by one to a list
- A forward propagation method to pass an input through the chain of modules, in the same order as they were appended

In [11]:
# using "childrne" class to access added modules

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, moduels in enumerate(args):
            self.add_module(str(idx), moduels)
    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X   

In [12]:
net = MySequential(nn.LazyLinear(256),nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [13]:
print(net)

MySequential(
  (0): Linear(in_features=10, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)


#### Executing Code in the Forward Propagation Method

- Perform arbitrary matchmatical operations, not simpely relying on predifned NNs
- sharing one linear layer and add some mathmatical operaitons

In [45]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [63]:
X = torch.rand((2, 20))
net = FixedHiddenMLP()
net(X)

tensor(-0.1759, grad_fn=<SumBackward0>)

In [66]:
# mixing various modules together
class NestMlp(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(20), nn.ReLU(),
                                nn.LazyLinear(50), nn.ReLU())
        self.linear = nn.LazyLinear(60)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMlp(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

C:\Users\shaohan.tian\scoop\apps\miniconda3\current\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor(-0.1548, grad_fn=<SumBackward0>)